importing os library for os level functions like folder structure

In [2]:
import os
print(os.listdir("../input"))
print(os.listdir("../"))

FileNotFoundError: ignored

importing standard libraries like numpy,pandas and matplotlib for data operations and plotting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# reading the data using pandas
traffic_2015 = pd.read_csv("../input/dot_traffic_2015.txt.gz")
traffic_2015_stations = pd.read_csv("../input/dot_traffic_stations_2015.txt.gz")


In [ ]:
#print the data columns for both datasets
print(traffic_2015.columns)
print(traffic_2015_stations.columns)

In [ ]:
lat_north = 33.506573
lat_south = 32.568381
long_west = 117.596359 
long_east = 116.081831
sd_traffic_2015_stations = traffic_2015_stations[(traffic_2015_stations["longitude"] <= long_west) \
                     & (traffic_2015_stations["longitude"] >= long_east) \
                     & (traffic_2015_stations["latitude"] <= lat_north) \
                     & (traffic_2015_stations["latitude"] >= lat_south)]
drop_columns = list(sd_traffic_2015_stations.columns.difference(['station_id','latitude','longitude']))
sd_traffic_2015_stations = sd_traffic_2015_stations.drop(drop_columns, axis = 1)
station_ids = sorted(sd_traffic_2015_stations["station_id"].unique())
for s in station_ids:
    latitude = 0
    longitude = 0
    locations = sd_traffic_2015_stations[sd_traffic_2015_stations["station_id"] == s].loc[:,"latitude":"longitude"]
    latitude = locations["latitude"].unique()
    longitude = locations["latitude"].unique()
    if latitude.size > 1:
        print("Station id",s,"has more than one latitude")
        print(latitude)
    elif longitude.size > 1:
        print("Station id",s,"has more than on longitude")
        print(longitude)
sd_traffic_2015 = traffic_2015[traffic_2015["station_id"].isin(station_ids)]
sd_traffic_2015_locations = pd.merge(sd_traffic_2015, sd_traffic_2015_stations,\
                                    left_on='station_id',right_on='station_id')
print(len(sd_traffic_2015_stations["station_id"].unique()))
print(len(sd_traffic_2015["station_id"].unique()))

print("IDs that were not found in the traffic dataset:")
for sd in station_ids:
    if sd not in sd_traffic_2015_locations["station_id"].unique():
        print(sd)

sd_station_ids = sd_traffic_2015_locations["station_id"].unique()

In [ ]:
import plotly

In [ ]:
#printing out the dataset shape and columns
print(sd_traffic_2015_locations.shape)
sd_traffic_2015_locations.columns

In [ ]:
sd_traffic_mean_year = pd.DataFrame()
tmp = list()
for station in sd_station_ids:
    tmp = np.mean(sd_traffic_2015_locations[sd_traffic_2015_locations["station_id"] == station].loc\
                                [:,"traffic_volume_counted_after_0000_to_0100":\
                                 "traffic_volume_counted_after_2300_to_2400"])
    sd_traffic_mean_year[station] = tmp
#plotting the data for better visualisation
fig = plt.figure(1,figsize=(35, 20))
ax = fig.gca()
ax.set_xticks(np.arange(0, 25, 1))
plt.title("Number of cars vs hours of the day for the year")
plt.xlabel("Hours of the day")
plt.ylabel("Number of cars")
plt.xlim((0,23))
for st in sd_station_ids:
    plt.plot(range(0,24,1),sd_traffic_mean_year[st])
plt.legend()
plt.grid()



We can see from the data that traffic patterns seem to be the same throughout 
the county. Around 4am traffic increases until about 9am. Traffic will begin 
decreasing around 5pm - 11pm. 